<a href="https://colab.research.google.com/github/satyopradana/capstone_api_satyo/blob/main/Proposal_DCD_21_Satyo_Pradana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PROPOSAL DCD 21 - Image Similarity

Satyo Pradana

1. Background

  Era digital sekarang yang sudah sangat berkembang, tidak akan pernah lepas akan kebutuhan teknologi termasuk internet didalamnya. Internet tidak hanya digunakan sebagai transfer data akan tetapi lebih dari itu. Data yang berada didalamnya menjadi sangat penting, sama pentingnya dengan minyak bumi atau kita lebih sering mendengar istilah "data is the new oil" jika suatu perusahaan dapat mengolah data dengan maksimal tidak menutup kemungkinan perusahaan akan meningkatkan baik performa maupun stabilitas perusahaan. Ditambah lagi dengan meningkatnya memory transfer data yang sangat besar dapat didominasi oleh format image dan video. Image dan video menjadi format dengan kebutuhan memory terbesar dibandingkan dengan format text. apalagi image dan video sekarang menjadi kebutuhan dasar manusia. Bahkan social media, e-commerce, media entertainment didominasi oleh image dan video dengan resolusi tinggi. Hal ini menjadi sumber atau hal yang harus diperhitungkan oleh sebuah perusahaan. Dengan meningkatnya kebutuhan data yang naik secara eksponensial membuat sebuah perusahaan memiliki peluang yang semakin besar pula untuk meraih keuntungan yang sebesar-besarnya.  

2. Problem Statement

  Dengan meningkatnya inovasi manusia dalam menciptakan ataupun mengembangkan sebuah produk yang bermanfaat dan unik membuat jenis serta nama dari suatu produk menjadi sangat banyak dalam mengkategorikannya. hal tersebut terkadang membuat konsumen bingung dengan nama produk tersebut. Image similarity merupakan metode untuk melihat kecocokan gambar terhadap dataset yang kita miliki.   

3. Problem Scope

   Pada project ini akan mengambil Fashion dataset pada Kaggle sebagai dataset training yang berjudul Farfetch Listings, namun tidak menutup kemungkinan akan ditambahkan dataset yang lain agar data train memiliki banyak pola sehingga akan mendapatkan lebih banyak jawaban. Model yang akan digunakan dalam project ini adalah model berupa pre train VGG16, Resnet 50, serta Xception sebagai pembanding antar model. Serta dari ketiga model tersebut akan dilakukan tuning sehingga akan mendapatkan hasil accuracy yang maksimal.


4. Project Idea

   Ide dari project ini adalah dengan menyediakan fitur input image pada suatu aplikasi sehingga memudahkan user untuk mencari product berdasarkan image similarity, akan memudahkan user tanpa harus mengetikkan keyword sehingga dapat meningkatkan efektifitas dan efisiensi serta akurat.

5. Output

   Dalam project ini program yang digunakan adalah Google Collaborator berbasis Python serta untuk visualisasi menggunakan Strimlit. Output akan ditampilkan dalam bentuk dashboard, dimana dashboard memiliki beberapa bentuk input method yang bisa dipilih oleh customer atau user berupa : upload file, link URL maupun direct photo dari camera. Output berupa tampilan image yang memiliki similarity dengan file input yang sudah dimasukan berdasarkan data training.

6. Business Impact

   Didalam project ini, Business Impact dapat dirasakan oleh beberapa pihak baik di sektor pemerintahan maupun swasta. Hal yang ditawarkan adalah perusahaan akan memiliki bank data atau dataset baru berupa images, dimana images tersebut dapat digunakan untuk memprediksi serta dapat menghasilkan keputusan bisnis yang tepat untuk kedepannya.   

7. Data Preparation
  
   Dataset Farfetch Listings memiliki 23 column dimana beberapa columnnya terdapat:
   availableSizes, yaitu ukuran yang tersedia
   brand.id, yaitu identitas dari sebuah brand
   brand.name, yaitu nama brand
   gender, gender yang memakai product tersebut
   hasSimilarProducts, apakah product tersebut memiliki kesamaan
   id,
   images.cutOut, foto product tanpa model
   images.model, foto product beserta model
   isCustomizable, apakah product bisa disesuaikan
   merchandiseLabel, labeling pada product
   merchandiseLabelField, tag pada product
   merchantId, Id pada merchant
   priceInfo.currencyCode, mata uang yang digunakan
   priceInfo.Finalprice, harga final product
   priceInfo.formatted.Finalprice, harga final produk beserta currency. Dalam tahapan ini dataset akan dilakukan cleansing terlebih dahulu, kemudian dilakukan data preprocessing.


In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from PIL import Image
from random import sample
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as vgg_preprocess
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as resnet_preprocess
from tensorflow.keras.applications.xception import Xception, preprocess_input as xception_preprocess
from tensorflow.keras.models import Model
from pathlib import Path
Image.LOAD_TRUNCATED_IMAGES = True
import requests
from io import BytesIO
import os

Melakukan import beberapa Library mulai dari pandas, numpy, matplotlib serta menggunakan tensorflow untuk melakukan modeling.

In [2]:
# Read the data files
listing_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/current_farfetch_listings.csv')

Melakukan read data berupa file csv

In [3]:
listing_data.head()

,Unnamed: 0,availableSizes,brand.id,brand.name,gender,hasSimilarProducts,id,images.cutOut,images.model,isCustomizable,...,priceInfo.currencyCode,priceInfo.discountLabel,priceInfo.finalPrice,priceInfo.formattedFinalPrice,priceInfo.formattedInitialPrice,priceInfo.initialPrice,priceInfo.installmentsLabel,priceInfo.isOnSale,shortDescription,stockTotal
0,0,NaN,8360,Miu Miu,women,True,13415450,https://cdn-images.farfetch-contents.com/13/41...,https://cdn-images.farfetch-contents.com/13/41...,False,...,SGD,NaN,1080,"$1,080","$1,080",1080,NaN,False,logo print strap sandals,130
1,1,"[{'scaleId': 0, 'size': 'XXS'}, {'scaleId': 0,...",2049368,Self-Portrait,women,True,13784566,https://cdn-images.farfetch-contents.com/13/78...,https://cdn-images.farfetch-contents.com/13/78...,False,...,SGD,20% Off,392,$392,$490,490,NaN,True,embroidered midi dress,14
2,2,NaN,15514,Fendi,women,True,13377441,https://cdn-images.farfetch-contents.com/13/37...,https://cdn-images.farfetch-contents.com/13/37...,False,...,SGD,NaN,1245,"$1,245","$1,245",1245,NaN,False,FendiMania sock style sneakers,68
3,3,"[{'scaleId': 0, 'size': 'One Size'}]",49486,A.P.C.,women,False,13744082,https://cdn-images.farfetch-contents.com/13/74...,https://cdn-images.farfetch-contents.com/13/74...,False,...,SGD,20% Off,308,$308,$385,385,NaN,True,top zip wallet,1
4,4,NaN,34624,Prada,women,True,13668256,https://cdn-images.farfetch-contents.com/13/66...,https://cdn-images.farfetch-contents.com/13/66...,False,...,SGD,NaN,1248,"$1,248","$1,248",1248,NaN,False,slingback 65 pumps,78


Menampilkan 5 dataset pertama untuk mengetahui jenis column apa saja yang terdapat dalam dataset.

In [4]:
listing_data.shape

(188817, 23)

Melihat dimensi dari dataset. Terdapat 188.817 baris serta 23 kolom.

In [5]:
  listing_data.drop('Unnamed: 0', axis=1, inplace=True)

Melakukan drop pada kolom yang tidak dibutuhkan.

In [7]:
listing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188817 entries, 0 to 188816
Data columns (total 22 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   availableSizes                   162711 non-null  object 
 1   brand.id                         188817 non-null  int64  
 2   brand.name                       188817 non-null  object 
 3   gender                           188817 non-null  object 
 4   hasSimilarProducts               188817 non-null  bool   
 5   id                               188817 non-null  int64  
 6   images.cutOut                    188817 non-null  object 
 7   images.model                     188817 non-null  object 
 8   isCustomizable                   188817 non-null  bool   
 9   merchandiseLabel                 56441 non-null   object 
 10  merchandiseLabelField            188817 non-null  object 
 11  merchantId                       188817 non-null  int64  
 12  pr

Setelah dilakukan checking, pada column priceInfo.installmentsLabel memiliki nilai 0 sehingga akan dilakukan dropping column

In [11]:
listing_data.drop('merchandiseLabel', axis=1, inplace=True)
listing_data.drop('availableSizes', axis=1, inplace=True)
listing_data['priceInfo.discountLabel'] = listing_data['priceInfo.discountLabel'].fillna(0)

In [12]:
listing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188817 entries, 0 to 188816
Data columns (total 19 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   brand.id                         188817 non-null  int64 
 1   brand.name                       188817 non-null  object
 2   gender                           188817 non-null  object
 3   hasSimilarProducts               188817 non-null  bool  
 4   id                               188817 non-null  int64 
 5   images.cutOut                    188817 non-null  object
 6   images.model                     188817 non-null  object
 7   isCustomizable                   188817 non-null  bool  
 8   merchandiseLabelField            188817 non-null  object
 9   merchantId                       188817 non-null  int64 
 10  priceInfo.currencyCode           188817 non-null  object
 11  priceInfo.discountLabel          188817 non-null  object
 12  priceInfo.finalP

Sudah dilakukan data wrangling sekarang data sudah bersih.

In [ ]:
def load_images():

    # Melakukan storing pada directory data
    cutout_img_dir = "../input/farfetch-listings/cutout-img/cutout"
    model_img_dir = "../input/farfetch-listings/model-img/model"

    # Melakukan listing nama pada dataset
    cutout_images = os.listdir(cutout_img_dir)
    model_images = os.listdir(model_img_dir)

    # Mengambil 10 random images pada cutout images dataset
    sample_cutout_images = sample(cutout_images,10)
    fig = plt.figure(figsize=(10, 5))

    print("==============Cutout Images==============")
    for i, img_name in enumerate(sample_cutout_images):
        plt.subplot(2, 5, i+1)
        img_path = os.path.join(cutout_img_dir, img_name)
        loaded_img = image.load_img(img_path)
        img_array = image.img_to_array(loaded_img, dtype='int')
        plt.imshow(img_array)
        plt.axis('off')

    plt.show()
    print()
    # Mengambil 10 random images pada model images dataset
    sample_model_images = sample(model_images,10)
    fig = plt.figure(figsize=(10,5))

    print("==============Model Images==============")
    for i, img_name in enumerate(sample_model_images):
        plt.subplot(2, 5, i+1)
        img_path = os.path.join(model_img_dir, img_name)
        loaded_img = image.load_img(img_path)
        img_array = image.img_to_array(loaded_img, dtype='int')
        plt.imshow(img_array)
        plt.axis('off')

    plt.show()